In [3]:
# 경로 추가
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')))

from app.utils import get_embedding_model


# 변수 선언
ROOT_PATH = '/Users/passion1014/project/langchain/rag_server'
INDEX_ID = "SAMPLE_simple" # VectorDB INDEX 명
FILE_NAME = f'{INDEX_ID}.simple' # 파일이름

# 임베딩 선언
embeddings = get_embedding_model()

In [9]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from app.vectordb import upload_vectordb
from langchain.docstore.document import Document
import re


# 파일 내용 로드 및 처리
text = upload_vectordb.load_and_detect_encoding(f'{ROOT_PATH}/data/{FILE_NAME}')


# 2. 의미별로 chunk로 나누기
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
# chunks = text_splitter.split_text(text)

table_pattern = r'### (.*?)\((.*?)\)\s*\{(.*?)\}'
chunks = re.findall(table_pattern, text, re.DOTALL)


# 각 chunk를 문서 객체로 변환
docs = text_splitter.create_documents(chunks)

# 각 chunk를 문서 객체로 변환 이 과정에서 각 청크를 개별 문서로 다룬다
docs = [Document(page_content=chunk) for chunk in chunks]

# FAISS 인덱스 생성 및 문서 추가
db = FAISS.from_documents(docs, embeddings)

# FAISS 인덱스 저장
db.save_local(f'{ROOT_PATH}/data/vector/{INDEX_ID}')


TypeError: expected string or bytes-like object, got 'tuple'